In [ ]:
import modules.m12_transform_data, modules.m21_model_selection, modules.m22_grid_search

In [ ]:
# parameters
algorithm_name = 'MF_SVD'
params = { 'n_factors' : 350, 'n_epochs' : 480, 'biased' : True, 'init_mean' : 0, 'init_std_dev' : 0.1}

In [ ]:
# get ratings set
df_ratings = modules.m12_transform_data.transform_data()
# get data to the model
data = modules.m21_model_selection.train_test_split(df_ratings=df_ratings)

In [ ]:
modules.m22_grid_search.predict(data, algorithm_name, params, False)

In [ ]:
# mlflow tracking
# mlflow parameters
# mlflow.set_tracking_uri("http://localhost:5000")
# mlflow.set_experiment("RecSys")

# with mlflow.start_run(run_name=algorithm_name):

#    with mlflow.start_run(run_name=today_date, nested=True):
#        mlflow.log_metric("foo", 12)
#        mlflow.log_artifact("run_summary", "run_summary_123")